In [43]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [44]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


In [45]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [46]:
lemmatizer = WordNetLemmatizer()
en_stop = set([lemmatizer.lemmatize(i) for i in get_stop_words('en')])

vocab  = open("wordsEn.txt").read().splitlines()
vocab = [lemmatizer.lemmatize(i)  for i in vocab]
vocab_set = set(vocab)


In [48]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()

        tokens = [lemmatizer.lemmatize(i) for i in word_tokenize(string)]

        #seperate words stuck together
        ntokens = []
        for i,token in enumerate(tokens):
            if token not in vocab_set:
                for j in range(2, len(token) - 1):
                    x = lemmatizer.lemmatize(token[:j])
                    y = lemmatizer.lemmatize(token[j:])
                    if x in vocab_set and j >= len(token)/2:
                        ntokens.append(x)
                        if y in vocab_set:
                                ntokens.append(y)                    
                        break               
                    elif y in vocab_set and j < len(token)/2:
                        tokens.append(y)
                        if x in vocab_set:
                                ntokens.append(x)                    
                        break                        
            else:
                ntokens.append(token)
        ntokens = [i for i in ntokens if i not in en_stop]   
        yield ntokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [49]:
%%time

T = texts()
dictionary_DCDRC = corpora.Dictionary(text for text in T.DCDRC())
dictionary_NCDRC = corpora.Dictionary(text for text in T.NCDRC())
dictionary_SCDRC = corpora.Dictionary(text for text in T.SCDRC())
dictionary_DelhiHC = corpora.Dictionary(text for text in T.DelhiHC())
dictionary_JharkhandHC = corpora.Dictionary(text for text in T.JharkhandHC())
dictionary_JodhpurHC = corpora.Dictionary(text for text in T.JodhpurHC())
dictionary_KolkataHC = corpora.Dictionary(text for text in T.KolkataHC())
dictionary_SupremeCourt = corpora.Dictionary(text for text in T.SupremeCourt())
# discard words occuring in less than 5 documents and in more than 50% 
dictionary_DCDRC.filter_extremes(keep_n=None)
dictionary_NCDRC.filter_extremes(keep_n=None) 
dictionary_SCDRC .filter_extremes(keep_n=None)
dictionary_DelhiHC.filter_extremes(keep_n=None) 
dictionary_JharkhandHC.filter_extremes(keep_n=None) 
dictionary_JodhpurHC.filter_extremes(keep_n=None) 
dictionary_KolkataHC.filter_extremes(keep_n=None) 
dictionary_SupremeCourt.filter_extremes(keep_n=None) 
# Merge all the dictionaries
dictionary = dictionary_DCDRC
dictionary.merge_with(dictionary_DelhiHC)
dictionary.merge_with(dictionary_NCDRC)
dictionary.merge_with(dictionary_SCDRC)
dictionary.merge_with(dictionary_SupremeCourt)
dictionary.merge_with(dictionary_JharkhandHC)
dictionary.merge_with(dictionary_JodhpurHC)
dictionary.merge_with(dictionary_KolkataHC)
dictionary.compactify()
dictionary.filter_extremes(keep_n=None)
# Need this after merging
dictionary.compactify()

2018-03-17 17:28:17,720 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-17 17:32:42,699 : INFO : adding document #10000 to Dictionary(17828 unique tokens: [u'unscientific', u'bypassed', u'pacemaker', u'middleman', u'progressively']...)
2018-03-17 17:36:37,594 : INFO : adding document #20000 to Dictionary(22806 unique tokens: [u'deferment', u'hanging', u'localized', u'regularize', u'disobeying']...)
2018-03-17 17:40:20,721 : INFO : adding document #30000 to Dictionary(26672 unique tokens: [u'deferment', u'clotted', u'hanging', u'localized', u'regularize']...)
2018-03-17 17:43:45,977 : INFO : adding document #40000 to Dictionary(29236 unique tokens: [u'deferment', u'clotted', u'hanging', u'trawling', u'localized']...)
2018-03-17 17:46:22,233 : INFO : adding document #50000 to Dictionary(30532 unique tokens: [u'deferment', u'pawnor', u'clotted', u'hanging', u'trawling']...)
2018-03-17 17:48:50,760 : INFO : adding document #60000 to Dictionary(31823 unique tokens: [u

2018-03-17 19:12:48,711 : INFO : keeping 10677 tokens which were in no less than 5 and no more than 1742 (=50.0%) documents
2018-03-17 19:12:48,722 : INFO : resulting dictionary: Dictionary(10677 unique tokens: [u'unscientific', u'yellow', u'narcotic', u'four', u'hanging']...)
2018-03-17 19:12:48,754 : INFO : discarding 11814 tokens: [(u'respondent', 46770), (u'justice', 29388), (u'dispute', 45880), (u'versus', 28935), (u'complaint', 44383), (u'cost', 39512), (u'per', 41735), (u'state', 46975), (u'order', 50724), (u'district', 46231)]...
2018-03-17 19:12:48,755 : INFO : keeping 18490 tokens which were in no less than 5 and no more than 28372 (=50.0%) documents
2018-03-17 19:12:48,772 : INFO : resulting dictionary: Dictionary(18490 unique tokens: [u'raining', u'unscientific', u'systematic', u'deferment', u'yellow']...)
2018-03-17 19:12:48,817 : INFO : discarding 12715 tokens: [(u'respondent', 31672), (u'ram', 30524), (u'high', 33958), (u'see', 24848), (u'paper', 22621), (u'allowed', 277

CPU times: user 1h 42min 51s, sys: 12.6 s, total: 1h 43min 3s
Wall time: 1h 44min 34s


In [50]:
len(dictionary)

32091

In [51]:
# This is BOW wrapper
c = my_corpus()
corpus = c.everything()

In [52]:
%%time

corpora.MmCorpus.serialize('z.mm', corpus)
######################################## load BOW
corpus = corpora.MmCorpus('z.mm')

2018-03-17 19:12:49,502 : INFO : storing corpus in Matrix Market format to z.mm
2018-03-17 19:12:49,503 : INFO : saving sparse matrix to z.mm
2018-03-17 19:12:50,144 : INFO : PROGRESS: saving document #0
2018-03-17 19:13:15,881 : INFO : PROGRESS: saving document #1000
2018-03-17 19:13:34,293 : INFO : PROGRESS: saving document #2000
2018-03-17 19:13:54,383 : INFO : PROGRESS: saving document #3000
2018-03-17 19:14:19,468 : INFO : PROGRESS: saving document #4000
2018-03-17 19:14:48,983 : INFO : PROGRESS: saving document #5000
2018-03-17 19:15:15,692 : INFO : PROGRESS: saving document #6000
2018-03-17 19:15:40,951 : INFO : PROGRESS: saving document #7000
2018-03-17 19:16:27,496 : INFO : PROGRESS: saving document #8000
2018-03-17 19:16:46,225 : INFO : PROGRESS: saving document #9000
2018-03-17 19:17:30,198 : INFO : PROGRESS: saving document #10000
2018-03-17 19:17:51,020 : INFO : PROGRESS: saving document #11000
2018-03-17 19:18:18,117 : INFO : PROGRESS: saving document #12000
2018-03-17 19

2018-03-17 19:55:16,924 : INFO : PROGRESS: saving document #122000
2018-03-17 19:55:41,292 : INFO : PROGRESS: saving document #123000
2018-03-17 19:56:03,458 : INFO : PROGRESS: saving document #124000
2018-03-17 19:56:32,665 : INFO : PROGRESS: saving document #125000
2018-03-17 19:56:57,556 : INFO : PROGRESS: saving document #126000
2018-03-17 19:57:19,217 : INFO : PROGRESS: saving document #127000
2018-03-17 19:57:35,194 : INFO : PROGRESS: saving document #128000
2018-03-17 19:57:51,455 : INFO : PROGRESS: saving document #129000
2018-03-17 19:58:06,003 : INFO : PROGRESS: saving document #130000
2018-03-17 19:58:17,630 : INFO : PROGRESS: saving document #131000
2018-03-17 19:58:30,505 : INFO : PROGRESS: saving document #132000
2018-03-17 19:58:42,986 : INFO : PROGRESS: saving document #133000
2018-03-17 19:58:57,381 : INFO : PROGRESS: saving document #134000
2018-03-17 19:59:07,666 : INFO : PROGRESS: saving document #135000
2018-03-17 19:59:18,463 : INFO : PROGRESS: saving document #13

2018-03-17 20:31:24,674 : INFO : PROGRESS: saving document #245000
2018-03-17 20:31:36,072 : INFO : PROGRESS: saving document #246000
2018-03-17 20:31:47,451 : INFO : PROGRESS: saving document #247000
2018-03-17 20:31:56,753 : INFO : PROGRESS: saving document #248000
2018-03-17 20:32:07,476 : INFO : PROGRESS: saving document #249000
2018-03-17 20:32:19,276 : INFO : PROGRESS: saving document #250000
2018-03-17 20:32:30,608 : INFO : PROGRESS: saving document #251000
2018-03-17 20:32:42,051 : INFO : PROGRESS: saving document #252000
2018-03-17 20:32:50,260 : INFO : PROGRESS: saving document #253000
2018-03-17 20:32:56,318 : INFO : PROGRESS: saving document #254000
2018-03-17 20:33:01,949 : INFO : PROGRESS: saving document #255000
2018-03-17 20:33:09,282 : INFO : PROGRESS: saving document #256000
2018-03-17 20:33:13,175 : INFO : PROGRESS: saving document #257000
2018-03-17 20:33:17,102 : INFO : PROGRESS: saving document #258000
2018-03-17 20:33:21,061 : INFO : PROGRESS: saving document #25

CPU times: user 1h 52min 28s, sys: 19.5 s, total: 1h 52min 47s
Wall time: 1h 52min 35s


In [53]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [54]:
%%time

# Save list of document lengths in the corpus
doc_len = []
for doc in corpus:
    temp = 0
    for word, freq in doc:
        temp += freq 
    doc_len.append(temp)


CPU times: user 4min 32s, sys: 612 ms, total: 4min 33s
Wall time: 4min 35s


In [55]:

import math
from six import iteritems
from six.moves import xrange

#     The variable k1 is a positive tuning parameter that calibrates the
#     document term frequency scaling. A k1 value of 0 corresponds to a binary
#     model (no term frequency), and a large value corresponds to using raw term
#     frequency. b is another tuning parameter (0 ≤ b ≤ 1) which determines
#     the scaling by document length: b = 1 corresponds to fully scaling the term
#     weight by the document length, while b = 0 corresponds to no length normalization


# BM25 parameters.
PARAM_K1 =1.2
PARAM_B = 0.75

# incase the idf is zero
EPSILON = 0.01


class BM25(object):

    def __init__(self, corpus):
        self.corpus_size = dictionary.num_docs
        s = 0
        for i in corpus:
            for x in i:
                s = s + x[1]
        s = float(s)    
        self.avgdl = s / self.corpus_size
        self.corpus = corpus
        self.df = dictionary.dfs
        self.idf = {}
        self.initialize()

    def initialize(self):
        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

    def get_score(self, query, index, average_idf):
        score = 0
        index_doc = dict(self.corpus[index])
        
        for word,freq in query:
            if word not in index_doc:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += freq*((idf * index_doc[word] * (PARAM_K1 + 1)
                      / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / self.avgdl))))
        return score

    def get_scores(self, query, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(query, index, average_idf)
            scores.append(score)
        return scores



In [56]:
%%time

bm25 = BM25(corpus)

CPU times: user 4min 28s, sys: 324 ms, total: 4min 29s
Wall time: 4min 28s


In [57]:
%%time

# need to pass it to the function that calculates the score
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 13.5 ms


In [58]:
%%time

results = copy.deepcopy(query_truth)

sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    sims = bm25.get_scores(dictionary.doc2bow(query), average_idf)
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
    print i
   
        

0
1
2
3
4
5
6
7
8
9
CPU times: user 20min 57s, sys: 16.9 s, total: 21min 14s
Wall time: 21min 14s


In [59]:
results

{'1': [['ConsumerCourt_DCDRC_100385', '1', 62.91237433159377, 1232],
  ['ConsumerCourt_DCDRC_106530', '0', 86.77204892950435, 283],
  ['ConsumerCourt_DCDRC_107608', '1', 49.995600620722314, 3131],
  ['ConsumerCourt_DCDRC_114291', '1', 108.56623293994285, 48],
  ['ConsumerCourt_DCDRC_114382', '1', 79.80503741864555, 438],
  ['ConsumerCourt_DCDRC_118185', '1', 73.5777723547313, 649],
  ['ConsumerCourt_DCDRC_130318', '0', 100.54180774325611, 105],
  ['ConsumerCourt_DCDRC_130570', '1', 99.99313929440646, 110],
  ['ConsumerCourt_DCDRC_131146', '1', 78.06804581227877, 513],
  ['ConsumerCourt_DCDRC_131717', '1', 67.10207035271178, 937],
  ['ConsumerCourt_DCDRC_131741', '1', 105.56710545046396, 65],
  ['ConsumerCourt_DCDRC_131818', '1', 100.97412209923232, 99],
  ['ConsumerCourt_DCDRC_131950', '0', 77.23696080814139, 542],
  ['ConsumerCourt_DCDRC_131972', '0', 126.53523519791024, 6],
  ['ConsumerCourt_DCDRC_132932', '0', 92.57043426101102, 193],
  ['ConsumerCourt_DCDRC_133592', '1', 102.546200

In [60]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'BM25.xlsx'
ws1 = wb.active
ws1.title = "K1 = 1.2 B = 0.75 EPS = 0.01 |"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

In [61]:
#Query wise (F score)

import pandas as pd

f = pd.ExcelFile('BM25.xlsx')

from scipy.stats import hmean



model_name = 'BM25'
places = 4
wb = load_workbook('BM25.xlsx')
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = 'BM25.xlsx')

In [62]:
#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile('BM25.xlsx')

from scipy.stats import hmean


model_name = 'BM25'
places = 4
wb = load_workbook('BM25.xlsx')
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):
        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = 'BM25.xlsx')

In [63]:
############################################################

In [ ]:
def Top_x(x, R):
    R = list(R)
    x_names = []
    for i in range(x):
        x_names.append(filenames[R.index(i + 1)].strip('.txt'))
    return x_names
    
    

In [ ]:
def score_stat(word, freq, index):
    index_doc = dict(bm25.corpus[index])
    word = dictionary.doc2bow([word])[0][0]
    idf = bm25.idf[word] if bm25.idf[word] >= 0 else EPSILON * average_idf
    score = freq*((idf * index_doc[word] * (PARAM_K1 + 1)
              / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / bm25.avgdl))))
    return (index_doc[word], score)

In [ ]:
wb = Workbook()
dest_filename = 'Temp.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)
def common_words(q, F):
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    index = filenames.index(F + '.txt')
    index_doc = dict(bm25.corpus[index])
    row = []
    for word, freq in  zip(q_words, q_freq):
        w = dictionary.doc2bow([word])[0][0]
        if w in index_doc: 
            f, s = score_stat(word, freq, index)
            row.append([word + ', ' + str(round(bm25.idf[w], 2)), freq, f,  round( (s*100)/sim_list[q][index], 1)])  
    row.sort(key = lambda x: (x[3],x[1],x[2]), reverse = True)
    wb = load_workbook('Temp.xlsx')
    ws = wb.create_sheet(title=str(q) + F)
    ws.append(['word,idf', 'Query freq', 'freq', '% score'])
    for i in row:
        ws.append(i)
    wb.save(filename = 'Temp.xlsx')


In [ ]:
f = open("toberead.txt", 'r') 
count = 0
q = 1
for i in f:
    if i == '\n':
        continue
    count = count + 1
    common_words(q, i.strip())
    if count % 5 == 0:
        q = q + 1


In [ ]:
for i in files_to_tokens(full_filenames[filenames.index(doc + '.txt')]):
    for j in i:
        print j

In [ ]:
doc = 'ConsumerCourt_SCDRC_30176'
corpus[]

In [ ]:
common_words(1, 'ConsumerCourt_DCDRC_41588')

In [ ]:
df = pd.DataFrame(index= None, columns=['words','pair'])


In [ ]:
df

In [ ]:
print df

In [ ]:
df.append(['lol', ('we','wew')])

In [ ]:
def uncommon_words(q, filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)

wb = load_workbook('Explore_BM25.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_uncom.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_uncom.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
#         for word, freq in  zip(q_words, q_freq):
#             if dictionary.doc2bow([word])[0][0] in index_doc: 
#                 f, s = score_stat(word, freq, index)
#                 row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
#             else:
#                 row.append('X')
        #ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'Explore_BM25_all.xlsx'
ws = wb.active
ws.title = "Blank"
#ws.append([(2.2,2.2)])
wb.save(filename = dest_filename)



wb = load_workbook('Explore_BM25_all.xlsx')

for q in range(1, 11):
    ws = wb.create_sheet(title="q" + str(q))
    #x_names = Top_x(10, rank_list[q])
    top_row = ['Doc Name', 'Rel', 'Doc Length', ' Rank', 'Score']
    sec_row = [' ']*5
    temp = []
    for word, freq in dictionary.doc2bow(queries[q]) :
        temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
    temp.sort(key=lambda x: (x[2],x[1]), reverse=True)
    ws.append(top_row + [i[0]  for i in temp])
    ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
    q_words = [i[0]  for i in temp]
    q_freq = [i[1]  for i in temp]
    
    #continue
    doc_list = [i[0] for i in query_truth[str(q)]] + Top_x(10, rank_list[q])
    doc_list = list((set(doc_list)))
    doc_list.sort(key = lambda doc : rank_list[q][filenames.index(doc + '.txt')])
    doc_row = []
    for doc in doc_list:
        temp  = [doc]
        rel_dic = dict(query_truth[str(q)])
        if doc in rel_dic:
            temp.append(rel_dic[doc])
        else:
            temp.append('Not')
        temp.append(doc_len[filenames.index(doc + '.txt')])
        temp.append(rank_list[q][filenames.index(doc + '.txt')])
        temp.append(round(sim_list[q][filenames.index(doc + '.txt')], 2))
        doc_row.append(temp)

    for row in doc_row:
        index = filenames.index(row[0] + '.txt')
        index_doc = dict(bm25.corpus[index])
        
        for word, freq in  zip(q_words, q_freq):
            if dictionary.doc2bow([word])[0][0] in index_doc: 
                f, s = score_stat(word, freq, index)
                row.append( str(f) + ', ' +  str(round( (s*100)/sim_list[q][index], 1) ))
            else:
                row.append('X')
        ws.append(row)
        
        sec_row = [' ']*5
        temp = []
        for word, freq in index_doc.iteritems():
            if dictionary[word] not in q_words:
                temp.append((dictionary[word], freq, round(bm25.idf[word], 2)))
        ws.append(sec_row + [i[0]  for i in temp])
        ws.append(sec_row + [str(i[1]) + ', ' + str(i[2]) for i in temp])
        ws.append([])        
wb.save(filename = dest_filename)